In [1]:
'''Мы разбили этот ноутбук на 2 ноутбука, в первом считаются одни метрики,
в следущем досчитывается другая метрика: face_emb_sim_between_real_img_and_generated_sticker'''


'Мы разбили этот ноутбук на 2 ноутбука, в первом считаются одни метрики,\nв следущем досчитывается другая метрика: face_emb_sim_between_real_img_and_generated_sticker'

In [2]:
#Первый ноутбук

In [ ]:
!pip install transformers accelerate gdown  pyiqa  -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.0/248.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s eta 0:00:00


In [ ]:
from scipy.spatial.qhull import QhullError
from scipy import spatial
spatial.QhullError = QhullError
from tqdm.auto import tqdm
import io
import glob
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import pyiqa
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import zlib
import requests
from transformers import AutoProcessor, AutoModel
import threading
import os
import shutil
import pyiqa
from torchvision.transforms.functional import pil_to_tensor
import cv2
from transformers import AutoImageProcessor
import numpy as np
import sys
import os
import numpy as np
import random
import torch
import torchvision
from PIL import Image, ImageDraw, ImageFont, ImageChops
device = 'cuda:0'

In [ ]:
clip_model_id = 'google/siglip-so400m-patch14-384'
iqa_model_id = 'topiq_iaa'
device = 'cuda:0'
SIZE = 512


aes_metric = pyiqa.create_metric(iqa_model_id).net.to('cuda:0')

clip_model = AutoModel.from_pretrained(clip_model_id).to('cuda:0')
processor = AutoProcessor.from_pretrained(clip_model_id)


IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)])

In [ ]:
def cosine_sim(x, y):
    return torch.nn.functional.cosine_similarity(x, y)

def aesthetic_score(x, device):
    x = transform(x).to(device)
    return aes_metric(x) / 10


def get_faces_score(img_sticker,img_orig, device='cuda:0'):

    img_orig_preprocessed = processor(images=img_orig, return_tensors="pt").to(device)

    img_sticker_preprocessed = processor(images=img_sticker, return_tensors="pt").to(device)

    prompt_is_sticker_preprocessed = processor(text='a sticker.', return_tensors="pt", padding="max_length").to(device)
    prompt_has_face_preprocessed = processor(text='a face.', return_tensors="pt", padding="max_length").to(device)



    with torch.no_grad():
        img_orig_emb = clip_model.get_image_features(**img_orig_preprocessed).to(device)
        img_sticker_emb = clip_model.get_image_features(**img_sticker_preprocessed).to(device)

        prompt_is_sticker_emb = clip_model.get_text_features(**prompt_is_sticker_preprocessed).to(device)
        prompt_has_face_emb = clip_model.get_text_features(**prompt_has_face_preprocessed).to(device)



    x1 = cosine_sim(img_orig_emb, img_sticker_emb).cpu().detach().numpy()[0]
    x3 = aesthetic_score(img_sticker, device).cpu().detach().numpy()[0][0]
    x4 = cosine_sim(img_sticker_emb, prompt_is_sticker_emb).cpu().detach().numpy()[0]
    x6 = aesthetic_score(img_orig, device).cpu().detach().numpy()[0][0]
    x8 = cosine_sim(img_sticker_emb, prompt_has_face_emb).cpu().detach().numpy()[0]


    data = {}
    data['sim_between_real_img_and_generated_sticker'] = x1
    data['aes_img'] = x6

    data['sim_between_generated_sticker_and_is_sticker_prompt'] = x4
    data['sim_between_generated_sticker_and_has_face_prompt'] = x8
    data['aes_sticker'] = x3
    return data



def parse_faces(paths):
    colnames = ['img', 'sticker',
       'sim_between_real_img_and_generated_sticker', 'aes_img',
       'sim_between_generated_sticker_and_is_sticker_prompt',
       'sim_between_generated_sticker_and_has_face_prompt', 'aes_sticker']
    data = {i:[] for i in colnames}
    for sticker_path, orig_path in tqdm(paths):
        data['img'].append(orig_path)
        data['sticker'].append(sticker_path)
        img_sticker = Image.open(sticker_path)
        img_orig = Image.open(orig_path)
        metrics = get_faces_score(img_sticker, img_orig)
        for i in colnames:
            if i in ['img', 'sticker']:
                continue
            data[i].append(metrics[i])
    return pd.DataFrame(data)

In [ ]:
import os


paths = []
for q in range(8):
    directory = f'/kaggle/input/fake-faces-dataset/fake_stickers_{q}/fake_stickers'
    for i in os.listdir(directory):
        if 'orig' not in i:
            paths.append(os.path.join(directory, i))

paths.sort(key=lambda x: int(x.split('/')[-1].split('_')[1]))


final_paths = []

for path in paths:
    idx = path.split('/')[-1].split('_')[1]
    orig_path = path.replace(f'{idx}_sticker', f'{idx}_orig')
    final_paths.append([path, orig_path])

print(final_paths[0])
print(final_paths[-1])
len(final_paths)

In [ ]:
from tqdm.auto import tqdm

result = parse_faces(final_paths)
result.to_csv('result.csv', index=False)

In [ ]:
#Следующий ноутбук

In [ ]:
!pip install gdown deepface -U
!gdown 1G93SzaMuTDDhK3ilklG4nbsxvDxazfcp

In [ ]:
import os
from tqdm.auto import tqdm
import pandas as pd
from deepface import DeepFace


paths = []
for q in range(8):
    directory = f'/kaggle/input/fake-faces-dataset/fake_stickers_{q}/fake_stickers'
    for i in os.listdir(directory):
        if 'orig' not in i:
            paths.append(os.path.join(directory, i))

paths.sort(key=lambda x: int(x.split('/')[-1].split('_')[1]))


final_paths = []

for path in paths:
    idx = path.split('/')[-1].split('_')[1]
    orig_path = path.replace(f'{idx}_sticker', f'{idx}_orig')
    final_paths.append([path, orig_path])

print(final_paths[0])
print(final_paths[-1])
len(final_paths)

In [ ]:
df = pd.read_csv('/kaggle/working/result_fakeds.csv')

In [ ]:
from tqdm.auto import tqdm

colnames = ['img', 'sticker', 'face_emb_sim_between_real_img_and_generated_sticker',
       'sim_between_real_img_and_generated_sticker', 'aes_img',
       'sim_between_generated_sticker_and_is_sticker_prompt',
       'sim_between_generated_sticker_and_has_face_prompt', 'aes_sticker']

data_x = {i:[] for i in colnames}


for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    for name in colnames:
        if name == 'face_emb_sim_between_real_img_and_generated_sticker':
            continue
        data_x[name].append(row[name])

    try:
        out = DeepFace.verify(img1_path = row['img'], img2_path = row['sticker'], detector_backend='retinaface')
        dist = out['distance']
    except ValueError:
        dist = 1

    data_x['face_emb_sim_between_real_img_and_generated_sticker'].append(1 - dist)

In [ ]:
df = pd.DataFrame(data_x)
df

In [ ]:
df.to_csv('fake-faces-dataset_cleaned.csv', index=False)